<a href="https://colab.research.google.com/github/kota-mizu/atmaCup-20/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install ydata-profiling

In [2]:
# ライブラリのインポート
import os, warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
from ydata_profiling import ProfileReport



warnings.filterwarnings("ignore")

# 日本語フォントを簡単に使う
!pip -q install japanize-matplotlib
import japanize_matplotlib

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data_path = '/content/drive/MyDrive/atmaCup#20/'

In [26]:
train_df  = pd.read_csv(data_path + 'train.csv')
test_df   = pd.read_csv(data_path + 'test.csv')
sample_submission_df = pd.read_csv(data_path + 'sample_submission.csv', header=None)  # header 無し
career_df = pd.read_csv(data_path + 'career.csv', index_col = 0)
dx_df = pd.read_csv(data_path + 'dx.csv', index_col = 0)
hr_df = pd.read_csv(data_path + 'hr.csv', index_col = 0)
overtime_df = pd.read_csv(data_path + 'overtime_work_by_month.csv', index_col = 0)
position_history_df = pd.read_csv(data_path + 'position_history.csv', index_col = 0)
udemy_activity_df = pd.read_csv(data_path + 'udemy_activity.csv', index_col = 0)

In [15]:
display(udemy_activity_df)

,コースID,コースタイトル,レクチャーもしくはクイズ,レクチャー/クイズID,レクチャー/クイズの題名,開始日,終了日,推定完了率%,最終結果（クイズの場合）,マーク済み修了,コースカテゴリー
社員番号,,,,,,,,,,,
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528090,企業オリジナル講座,2022/4/11 10:10,2022/4/11 10:10,100.0,0.0,True,企業オリジナル講座
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528090,企業オリジナル講座,2022/4/11 10:11,2022/4/11 10:11,100.0,100.0,True,企業オリジナル講座
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528100,企業オリジナル講座,2022/4/11 10:26,2022/4/11 10:26,100.0,0.0,True,企業オリジナル講座
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528100,企業オリジナル講座,2022/4/11 10:27,2022/4/11 10:27,100.0,100.0,True,企業オリジナル講座
-1sqs0GXzpPJuAVKHUUFgg==,4615016,企業オリジナル講座,Quiz,5528102,企業オリジナル講座,2022/4/11 10:21,2022/4/11 10:21,100.0,100.0,True,企業オリジナル講座
...,...,...,...,...,...,...,...,...,...,...,...
zxY0Eflwm1tYj1Wt6vo_1g==,5264112,企業オリジナル講座,Video_lecture,37311144,企業オリジナル講座,2023/6/14 9:56,2023/6/14 9:56,100.0,NaN,True,企業オリジナル講座
zxY0Eflwm1tYj1Wt6vo_1g==,5264112,企業オリジナル講座,Video_lecture,37311150,企業オリジナル講座,2023/6/14 9:57,2023/6/14 9:57,100.0,NaN,True,企業オリジナル講座
zxY0Eflwm1tYj1Wt6vo_1g==,6098315,企業オリジナル講座,Video_lecture,45012691,企業オリジナル講座,2024/9/27 15:39,2024/9/27 16:17,100.0,NaN,True,企業オリジナル講座


In [27]:
applicant_counts = train_df['社員番号'].value_counts()
applicant_counts

,count
社員番号,
zuplFpzBoM4c1dFy5HPXqg==,6
-2Sq3E0WkZj8pL7jxdL3Cg==,6
-4taxxVbT1nU-J5fHWmDfQ==,6
-6LqZqVuZIYsbdzQPYGopw==,6
-EtuCRccKFQgi3UfXRvRkA==,6
...,...
-jxlIDFHFEkeLp7PXL7f5g==,6
-ibwaU2D_eur2L7vAlCmEg==,6
-eVRJ1yPWXf_0ljndBfpTA==,6


In [28]:
train_df[train_df['社員番号'] == "zuplFpzBoM4c1dFy5HPXqg=="]

,社員番号,category,target
7332,zuplFpzBoM4c1dFy5HPXqg==,コンテンツ・サービス・デザイン,0
7333,zuplFpzBoM4c1dFy5HPXqg==,コーポレート管理部門/技術・データ・BPR,0
7334,zuplFpzBoM4c1dFy5HPXqg==,プロダクトマネジメント,0
7335,zuplFpzBoM4c1dFy5HPXqg==,マーケティング,0
7336,zuplFpzBoM4c1dFy5HPXqg==,事業企画・開発・研究,0
7337,zuplFpzBoM4c1dFy5HPXqg==,営業,0


In [32]:
    # 2. 応募した記録（target == 1）のみを抽出する
    applicants_df = train_df[train_df['target'] == 1]

    # 3. 抽出したデータから、ユニークな社員番号の数を数える
    num_unique_applicants = applicants_df['社員番号'].nunique()
    print(num_unique_applicants)
    print(train_df['社員番号'].nunique())

255
1223
